In [1]:
import pandas as pd
import numpy as np
import datetime

In [2]:
sonoma = pd.read_csv("Animal_Shelter_Intake_and_Outcome.csv")

### Summary

Data cleaning:
- drop unnecessary columns
- change index
- clean columns with .str(), .split(), .fillna(), .apply() [with context], 
- rename columns to combine with other datasets
- skip rows?

This data:
- WeekYear format for dates (mid 2000s - present)
- CountOf...
- intakes/outtakes statistics
- (Optional)

Observations:
- 884 breeds
- outcome_jurisdicition has inconsistent cities/towns (see below)

In [3]:
# replace column names: " " to "_" and change to lowercase
def remove_space(column_name):
    return column_name.lower().replace(' ','_')
sonoma = sonoma.rename(remove_space, axis = "columns")

In [4]:
sonoma = sonoma.loc[sonoma.type == "DOG"]

In [5]:
# "location" values are of the form zipcode(latitude,longitude)
# this function keeps only the zipcode of outcome jurisdiction
# there is already an "outcome_zip_code" column with float, I'm too lazy to modify it and will drop this column
def remove_lat_long(column_name):
    return str(column_name)[:5]
sonoma["zipcode"] = sonoma["location"].apply(remove_lat_long)

In [6]:
# outcome_jurisdicition has inconsistent cities/towns (see below)
def remove_ast(column_name):
    if str(column_name) == "UNKNOWN":
        return np.nan
    elif column_name == np.nan:
        return np.nan
    else:
        return str(column_name).replace("*","")
sonoma["outcome_jurisdiction"] = sonoma["outcome_jurisdiction"].apply(remove_ast)
sonoma["outcome_jurisdiction"].value_counts()

outcome_jurisdiction
SANTA ROSA       5147
COUNTY           4689
OUT OF COUNTY    2228
nan              1592
WINDSOR           882
ROHNERT PARK      427
PETALUMA          340
HEALDSBURG        120
CLOVERDALE        105
SEBASTOPOL         80
COTATI             59
SONOMA             37
Name: count, dtype: int64

In [7]:
sonoma.columns

Index(['name', 'type', 'breed', 'color', 'sex', 'size', 'date_of_birth',
       'impound_number', 'kennel_number', 'animal_id', 'intake_date',
       'outcome_date', 'days_in_shelter', 'intake_type', 'intake_subtype',
       'outcome_type', 'outcome_subtype', 'intake_condition',
       'outcome_condition', 'intake_jurisdiction', 'outcome_jurisdiction',
       'outcome_zip_code', 'location', 'count', 'zipcode'],
      dtype='object')

In [8]:
# outcome_subtype has a lot of noisy data
to_drop = ["name","type","breed","color","impound_number","kennel_number","animal_id","intake_jurisdiction","outcome_jurisdiction","outcome_zip_code","location","count"]
sonoma = sonoma.drop(to_drop,axis =1)

In [9]:
sonoma = sonoma.replace({np.nan: np.nan,"NaN": np.nan, "nan": np.nan, "Unknown": np.nan,"UNKNOWN":np.nan})

In [10]:
# function that takes date to weekyear: '01/08/2024' -> '2-2024'
def weekyear_converter(column_name):
    return str(pd.Timestamp(column_name).weekofyear) + "-" + str(pd.Timestamp(column_name).year)

sonoma["intake_date_week_year"] = sonoma["intake_date"].apply(weekyear_converter)
sonoma["outcome_date_week_year"] = sonoma["outcome_date"].apply(weekyear_converter)

In [11]:
sonoma.head()

,sex,size,date_of_birth,intake_date,outcome_date,days_in_shelter,intake_type,intake_subtype,outcome_type,outcome_subtype,intake_condition,outcome_condition,zipcode,intake_date_week_year,outcome_date_week_year
1,Male,MED,NaN,12/04/2023,01/08/2024,35,STRAY,FLD_STRAY,EUTHANIZE,AGGRESSIVE,NaN,UNTREATABLE,NaN,49-2023,2-2024
2,Neutered,LARGE,07/27/2016,07/16/2019,08/30/2019,45,STRAY,FIELD,TRANSFER,HSSC,HEALTHY,HEALTHY,95407,29-2019,35-2019
3,Female,MED,NaN,11/28/2023,01/08/2024,41,STRAY,FLD_STRAY,EUTHANIZE,BEH SEVERE,NaN,UNTREATABLE,NaN,48-2023,2-2024
4,Female,LARGE,11/11/2021,11/11/2023,01/08/2024,58,STRAY,FIELD,EUTHANIZE,AGGRESSIVE,NaN,UNTREATABLE,NaN,45-2023,2-2024
5,Spayed,SMALL,06/08/2021,01/05/2024,01/08/2024,3,STRAY,OVER THE COUNTER,RETURN TO OWNER,NaN,NaN,HEALTHY,93927,1-2024,2-2024


In [18]:
# create CountOf.....

# Example
sonoma_count = sonoma["intake_date_week_year"].value_counts().rename_axis("week_year").to_frame("intakes")
sonoma_count["outcomes"] = sonoma["outcome_date_week_year"].value_counts().astype('Int64')
sonoma_count = sonoma_count.fillna(0)